In [1]:
!git clone https://github.com/maiphong0411/code_generation.git

Cloning into 'code_generation'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 101 (delta 36), reused 85 (delta 21), pack-reused 0
Receiving objects: 100% (101/101), 12.05 MiB | 29.11 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [2]:

!pip install -q peft transformers datasets
!pip install -r code_generation/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.7
    Uninstalling dill-0.3.7:
      Successfully uninstalled dill-0.3.7
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.15
    Uninstalling multiprocess-0.70.15:
      Successfully uninstalled multiprocess-0.70.15
  Attempting uninstall: peft
    Found existing installation: peft 0.5.0
    Uninstalling peft-0.5.0:
      Suc

In [3]:
path_data = "/kaggle/working/code_generation/test.json"
import json
with open(path_data, "r") as file:
  data = json.load(file)
     
data['output'][str(0)]

'# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum'

In [4]:
len(data['instruction'])

1861

In [5]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup

peft_model_id = "phongmt184172/bloom_lora"

config = PeftConfig.from_pretrained(peft_model_id)
print(config.base_model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


bigscience/bloom-1b7


In [6]:
print(model)
model.cuda()
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 2048)
        (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): Linear(
                in_features=2048, out_features=6144, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=6144, bias=False)
                )
                (lora_embedding_A): ParameterDict(

In [9]:

from tqdm import tqdm

max_sample = len(data['instruction'])

for i in tqdm(range(max_sample)):
    instruction = data['instruction'][str(i)]
    input = data['input'][str(i)]
    output_gt = data['output'][str(i)]
    
    template = {"description": "Template used by Alpaca-LoRA.",
                "prompt_input": f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
                "prompt_no_input": f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
                "response_split": "### Response:"}
    if input:
        prompt = template['prompt_input']
    else:
        prompt = template['prompt_no_input']

    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
#     input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.to('cpu')
    outputs = model.generate(input_ids=input_ids, max_new_tokens=600, do_sample=True, top_p=0.9,temperature=0.3)

#     print(f"{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
    out = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
    out = out.replace("\n", "\\n")
    
    with open('predict.txt', 'a', encoding='utf-8') as file:
        file.write(out + '\n')
    
    output_gt = output_gt.replace("\n", "\\n")
    with open('label.txt', 'a', encoding='utf-8') as file2:
        file2.write(output_gt + '\n')

100%|██████████| 1861/1861 [1:12:18<00:00,  2.33s/it]
